# HOW TO: Generate Fake Data with R (Using Jupyter Notebook)
`--------------------------------------------------`

**___________________________Featuring: Mr Fugu Data Science____**


[Find me on Github](https://github.com/MrFuguDataScience)

`----------------------------------------------------------`

`It appears that while I have packages loaded into R-Studio, do not coincide with Jupyter Notebook. I tried to search Anaconda installs but could not find them. Hence, we are doing this, if it becomes a habit I will create a function call.`

*Also, `Tidyverse` takes several minutes alone due to dependencies*

[rpython](https://cran.r-project.org/web/packages/rPython/index.html)| [reticulate](https://cran.r-project.org/web/packages/reticulate/vignettes/calling_python.html) | [RPostgreSQL](https://cran.r-project.org/web/packages/RPostgreSQL/index.html) | [dbConnect](https://www.rdocumentation.org/packages/DBI/versions/0.5-1/topics/dbConnect) | [randomNames](https://cran.r-project.org/web/packages/randomNames/vignettes/randomNames.html)

In [1]:
# install.packages("generator")
# install.packages("dbConnect")
# install.packages("tidyverse")
# install.packages("rlang")
# install.packages("RPostgreSQL")
# install.packages("rPython")
# install.packages("randomNames")
# install.packages("reticulate")

In [2]:
library(generator) # fake personal information
library(knitr) # help run code
library(markdown) # create markdown files i.e. pdf
library(dbConnect) # connect to databases
library(DBI) # R database interface
library(tidyverse) #if you parse files
library(dplyr)
library(RPostgreSQL) # run psql instance
library(rPython)  #Run Python code, make function calls, assign and retrieve variables, etc. from R.
library(randomNames)  #[this is not updated in R-studio] as of Apr 1,2020 but works here :) 
library(reticulate)  # interface with python modules,classes, functions

Loading required package: RMySQL
Loading required package: DBI
Loading required package: gWidgets
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::id()     masks gWidgets::id()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘RPostgreSQL’

The following object is masked from ‘package:RMySQL’:

    dbApply

Loading required package: RJSONIO


In [3]:
#Users info:

crdt_crd<-generator::r_credit_card_numbers(n=6000)
email<-generator::r_email_addresses(n=6000)

dob<-generator::r_date_of_births(n=6000)

first_name<-randomNames(n=6000,return.complete.data = FALSE,which.names = "first")
last_name<-randomNames(n=6000,return.complete.data = FALSE,which.names = "last")
primary_phone_number<-generator::r_phone_numbers(n=6000)


In [4]:
user_info<-list(crdt_crd,email,first_name,last_name,primary_phone_number)
data.frame(matrix((user_info),nrow=length(user_info)))


matrix..user_info...nrow...length.user_info..


In [5]:
users<-as.data.frame(matrix(unlist(cbind(user_info)),ncol = 5),as.factor=FALSE)
users<-users[0:5826,]
# users
names(users)[1]<-paste("credit_card")
names(users)[2]<-paste("email")
names(users)[3]<-paste("first_name")
names(users)[4]<-paste("last_name")
names(users)[5]<-paste("primary_phone_number")
head(users)

credit_card,email,first_name,last_name,primary_phone_number
7042-4573-9785-2864,louk@dqahzvo.teo,Austin,Chavez,6435319854
5417-7284-5499-1944,yuz@wr.zla,Jessica,Leighton,8654534678
1442-8536-6945-5053,mnpjz@bxk.ouk,Hannah,Khuu,6278961573
7224-3381-8194-8672,nhodtbykw@wxfabjedg.zkp,Wayne,Lenahan,2919754893
8239-3630-7593-9946,pqancf@ra.mvh,Cristian,Buckendorf,5313793758
4011-1512-7082-6859,r@ulrp.jzy,Shawn,Green,8594274951


# Before we proceed: there is a caveat and I need to discuss, when creating random (pseudo) random data you need to look for duplicates.

+ There are duplicate names (first and last)
    + `I AM ONLY CONCERNED IF THE PAIR OF NAMES COMES UP MORE THAN 1 TIME`:

In [6]:
# Here is an issue when you create random data!

sum(duplicated(users$credit_card))
sum(duplicated(users$email))
sum(duplicated(users$last_name))
sum(duplicated(users$primary_phone_number))
sum(duplicated(users$first_name))

[1] 0

[1] 0

[1] 2236

[1] 0

[1] 3003

# There are 13 matches to (first,last) names which need to be removed from our database! 
+ It will depend what we want as a qualifier, we can keep them and base our data off of an `ID` which will be the `primary key` for a `RDMS`, and figure out what to use for a `NoSQL`. 

+ While this can create problems, you have to understand people have same names and are different people and need to establish what other data will coincide causing issues. 

In [7]:
# users %>% group_by(first_name, last_name) %>% summarize(time = max(time, na.rm = T))
users[duplicated(users[c(3, 4)]),] 
sum(duplicated(users[,3:4]))

# users[duplicated(users[,3:4]),]


,credit_card,email,first_name,last_name,primary_phone_number
1815,8475-1845-3511-5060,zjewlrfaub@ura.kfj,Kevin,Williams,8911263791
2675,6950-4496-4201-2608,fjoqudc@rchwyk.gwa,Joshua,Lee,2874564531
3004,2187-2851-5169-5405,wnsxyeaghq@f.kyp,Jorge,Yazzie,2467438641
3672,2217-7750-1578-5829,vlwfuiep@fwrbdc.hpc,Alexis,Jones,9626146257
3685,3450-3587-2522-7434,hpiogzvdk@r.xug,Austin,Smith,4296922563
3791,1855-9289-3564-6859,hl@hmqlijubv.zbh,Brandon,Thomas,5391726815
3908,5038-2287-6765-2657,sjiymcbf@ipwylesjhn.wno,Jose,Cornelius,5871565468
4253,6679-2439-6885-8657,pvtymq@mqlpfoa.tkq,Amanda,Martinez,6277434798
4295,6540-6264-3716-3110,tq@vgjc.lwn,Nicole,Martinez,9234714921
4316,2988-3354-6571-4187,nsrgi@pqd.itw,Madison,Garcia,4959566495


[1] 18

In [27]:
# I am accessing Python module [OS] and calling with R formatting:

#os <- import("os")
#os$listdir(".")
# write(users,'fake_users.csv')
# typeof(users)

data.frame(matrix((users),nrow=length(users)))
usr<-as.data.frame(matrix(unlist(cbind(users)),ncol = 5),as.factor=FALSE)

is.data.frame(usr)
is.list(usr)

matrix..users...nrow...length.users..


[1] TRUE

[1] TRUE


# Time to Dance because we are going to send these data from R --> Mysql

**Run when getting to this line:**
<!-- ![dance_time](giphy.gif) -->

In [31]:
write.csv(usr,'fake_users_R.csv')

# Citations:

https://stackoverflow.com/questions/42459423/cannot-install-r-packages-in-jupyter-notebook

https://stackoverflow.com/questions/6081439/changing-column-names-of-a-data-frame

https://cran.r-project.org/web/packages/reticulate/vignettes/calling_python.html

https://giphy.com/gifs/dance-alien-ufo-Z9WQLSrsQKH3uBbiXq (NO Copyrights!)

https://programminghistorian.org/en/lessons/getting-started-with-mysql-using-r

https://jagg19.github.io/2019/05/mysql-r/

https://cran.r-project.org/doc/manuals/r-devel/R-data.html#DBI